# SO-ARM 机械臂校准和遥操作

本教程介绍如何校准 SO-ARM 机械臂，并测试主从臂的遥操作功能。

**前置条件：**
- ✅ 已完成 `00_installation_and_setup.ipynb`
- ✅ 两个机械臂已连接并识别
- ✅ 所有舵机 ID 已正确配置

**本教程内容：**
1. 校准 Follower Arm（从动臂）
2. 校准 Leader Arm（主动臂）
3. 测试遥操作功能
4. 添加摄像头支持
5. 测试完整系统

**参考：** [Seeed Studio LeRobot Wiki - 校准](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/#_4)

---
## 📋 步骤 0: 准备工作

### 硬件检查

在开始校准前，确保：

1. **Follower Arm（从动臂）**
   - 电源已接通
   - USB 线连接到 /dev/ttyACM0
   - 所有舵机 ID 已配置 (1-7)
   - 机械臂在安全位置（周围无障碍物）

2. **Leader Arm（主动臂）**
   - 电源已接通
   - USB 线连接到 /dev/ttyACM1
   - 所有舵机 ID 已配置 (1-7)
   - 可以自由移动（无阻力）

### 什么是校准？

**校准 (Calibration)** 是建立机械臂物理位置与控制系统之间对应关系的过程。

- **目的：** 让系统知道每个舵机的零点位置和运动范围
- **重要性：** 校准不准确会导致运动不精确或损坏硬件
- **频率：** 每次组装后、更换舵机后、或精度下降时需要重新校准

---
## 🔧 步骤 1: 校准 Follower Arm（从动臂）

### 1.1 启动校准程序

In [ ]:
print("在终端运行以下命令开始校准 Follower Arm：")
print()
print("lerobot-calibrate \\")
print("  --robot.type=so101_follower \\")
print("  --robot.port=/dev/ttyACM0 \\")
print("  --robot.id=my_so101_follower")
print()
print("参数说明：")
print("  --robot.type: 机械臂类型 (so101_follower)")
print("  --robot.port: 串口设备路径")
print("  --robot.id: 给机械臂起一个标识名称")
print()
print("⚠️ 注意：--robot.id 会被保存，后续所有操作都要使用相同的 ID！")

### 1.2 校准步骤

校准程序会引导您完成以下步骤：

#### 第1步：移动到初始位置

```
Move arm to a position where it is easy to move each motor.
When done, press Enter
```

**操作：**
- 手动将机械臂移动到一个舒适的位置
- 确保所有关节都可以自由转动
- 建议采用半伸展姿态
- 按 Enter 继续

#### 第2步：逐个关节旋转

对于每个关节（ID 1-7），程序会提示：

```
Rotate the motor 'shoulder_pan' clockwise and
counter-clockwise by a quarter turn. 
Press Enter when done.
```

**操作：**
1. 找到对应的关节（根据提示的名称）
2. 顺时针旋转约 90度
3. 逆时针旋转约 90度
4. 回到中间位置
5. 按 Enter 继续

**关节对应关系：**
- `shoulder_pan` (ID 1): 底座旋转
- `shoulder_lift` (ID 2): 肩部抬升
- `elbow_flex` (ID 3): 肘部弯曲
- `wrist_flex` (ID 4): 腕部弯曲
- `wrist_roll` (ID 5): 腕部旋转
- `gripper` (ID 6): 夹爪开合

#### 第3步：保存校准数据

```
Calibration is done! Calibration file saved to:
.cache/calibration/my_so101_follower.json
```

**重要：** 校准数据会保存到本地，不要删除这个文件！

### 1.3 验证校准结果

In [ ]:
from pathlib import Path
import json

# 检查校准文件是否存在
calibration_file = Path.home() / ".cache" / "calibration" / "my_so101_follower.json"

if calibration_file.exists():
    print(f"✓ 校准文件已找到: {calibration_file}")
    print()
    
    # 读取并显示校准数据
    with open(calibration_file, 'r') as f:
        calib_data = json.load(f)
    
    print("校准数据摘要：")
    print(f"  关节数量: {len(calib_data.get('joints', {}))}")
    print(f"  时间戳: {calib_data.get('timestamp', 'N/A')}")
    print()
    print("各关节校准范围：")
    for joint_name, joint_data in calib_data.get('joints', {}).items():
        print(f"  {joint_name}: {joint_data.get('homing_offset', 'N/A')}")
else:
    print("✗ 未找到校准文件")
    print("请先运行校准程序：lerobot-calibrate ...")

---
## 🎮 步骤 2: 校准 Leader Arm（主动臂）

**Leader Arm 的校准过程与 Follower Arm 完全相同。**

In [ ]:
print("在终端运行以下命令开始校准 Leader Arm：")
print()
print("lerobot-calibrate \\")
print("  --teleop.type=so101_leader \\")
print("  --teleop.port=/dev/ttyACM1 \\")
print("  --teleop.id=my_so101_leader")
print()
print("⚠️ 注意：")
print("  - 使用 --teleop 参数而不是 --robot")
print("  - ID 名称要与 Follower 不同")
print("  - 确认使用正确的串口 (ttyACM1)")
print()
print("重复 Follower Arm 的校准步骤：")
print("  1. 移动到初始位置")
print("  2. 逐个旋转关节")
print("  3. 保存校准数据")

---
## 🚀 步骤 3: 测试遥操作（无摄像头）

### 3.1 基础遥操作测试

完成两个机械臂的校准后，可以测试主从控制功能。

In [ ]:
print("在终端运行以下命令测试遥操作：")
print()
print("lerobot-teleoperate \\")
print("  --robot.type=so101_follower \\")
print("  --robot.port=/dev/ttyACM0 \\")
print("  --robot.id=my_so101_follower \\")
print("  --teleop.type=so101_leader \\")
print("  --teleop.port=/dev/ttyACM1 \\")
print("  --teleop.id=my_so101_leader")
print()
print("预期行为：")
print("  - 移动 Leader Arm，Follower Arm 应该同步跟随")
print("  - 延迟应该很小（< 100ms）")
print("  - 动作应该流畅自然")
print()
print("测试要点：")
print("  1. 缓慢移动每个关节，检查跟随精度")
print("  2. 测试全范围运动")
print("  3. 测试夹爪开合")
print("  4. 检查是否有卡顿或异常")
print()
print("⚠️ 按 Ctrl+C 可以随时停止遥操作")

### 3.2 遥操作故障排查

#### 问题 1: Follower Arm 不跟随

**可能原因：**
- 校准数据不正确
- 串口连接断开
- ID 名称不匹配

**解决方法：**
1. 检查校准文件是否存在
2. 确认使用的 ID 与校准时相同
3. 重新校准

#### 问题 2: 动作延迟很大

**可能原因：**
- USB Hub 导致延迟
- 串口通信速率低
- 系统资源不足

**解决方法：**
1. 使用 USB 直连（不要用 Hub）
2. 关闭其他占用 CPU 的程序
3. 检查串口波特率设置

#### 问题 3: 关节抖动或卡顿

**可能原因：**
- 舵机过热
- 机械阻力大
- 电源供电不足

**解决方法：**
1. 让机械臂休息冷却
2. 检查机械连接是否顺畅
3. 确认电源输出电流足够 (≥4A)

---
## 📹 步骤 4: 添加摄像头

### 4.1 检测可用摄像头

In [ ]:
import cv2

print("检测可用摄像头...")
print()

available_cameras = []

for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        ret, frame = cap.read()
        if ret:
            h, w = frame.shape[:2]
            print(f"✓ 摄像头 {i}: {w}x{h}")
            available_cameras.append(i)
        cap.release()

print()
if available_cameras:
    print(f"找到 {len(available_cameras)} 个摄像头: {available_cameras}")
    print()
    print("推荐配置：")
    print(f"  - 主摄像头 (front): 索引 {available_cameras[0]}")
    if len(available_cameras) > 1:
        print(f"  - 腕部摄像头 (wrist): 索引 {available_cameras[1]}")
else:
    print("✗ 未检测到摄像头")
    print("请连接 USB 摄像头后重试")

### 4.2 测试摄像头画面

In [ ]:
import cv2
import matplotlib.pyplot as plt

# 选择要测试的摄像头
camera_index = 0  # 修改为您的摄像头索引

print(f"测试摄像头 {camera_index}...")

cap = cv2.VideoCapture(camera_index)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)

if cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 7))
        plt.imshow(frame_rgb)
        plt.title(f"摄像头 {camera_index} - {frame.shape[1]}x{frame.shape[0]}")
        plt.axis('off')
        plt.show()
        
        print("✓ 摄像头测试成功！")
    else:
        print("✗ 无法读取画面")
    cap.release()
else:
    print(f"✗ 无法打开摄像头 {camera_index}")

### 4.3 带摄像头的遥操作

#### 单摄像头配置

In [ ]:
print("在终端运行以下命令（单摄像头）：")
print()
print("lerobot-teleoperate \\")
print("  --robot.type=so101_follower \\")
print("  --robot.port=/dev/ttyACM0 \\")
print("  --robot.id=my_so101_follower \\")
print("  --robot.cameras='{front: {type: opencv, index_or_path: 0, \\")
print("    width: 640, height: 480, fps: 30, fourcc: \"MJPG\"}}' \\")
print("  --teleop.type=so101_leader \\")
print("  --teleop.port=/dev/ttyACM1 \\")
print("  --teleop.id=my_so101_leader")
print()
print("参数说明：")
print("  front: 摄像头名称（自定义）")
print("  type: opencv（使用 OpenCV 驱动）")
print("  index_or_path: 摄像头索引或设备路径")
print("  width/height: 分辨率")
print("  fps: 帧率")
print("  fourcc: 视频编码格式（MJPG 兼容性好）")

#### 双摄像头配置（可选）

In [ ]:
print("如果有两个摄像头，可以同时使用：")
print()
print("lerobot-teleoperate \\")
print("  --robot.type=so101_follower \\")
print("  --robot.port=/dev/ttyACM0 \\")
print("  --robot.id=my_so101_follower \\")
print("  --robot.cameras='{")
print("    front: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30, fourcc: \"MJPG\"},")
print("    wrist: {type: opencv, index_or_path: 1, width: 640, height: 480, fps: 30, fourcc: \"MJPG\"}")
print("  }' \\")
print("  --teleop.type=so101_leader \\")
print("  --teleop.port=/dev/ttyACM1 \\")
print("  --teleop.id=my_so101_leader")
print()
print("⚠️ 注意：多摄像头会增加 CPU 负载，可能导致帧率下降")

---
## ✅ 步骤 5: 完整系统测试

### 测试检查清单

完成所有配置后，进行以下测试：

In [ ]:
checklist = [
    "Follower Arm 已成功校准",
    "Leader Arm 已成功校准",
    "无摄像头时遥操作正常",
    "Leader 移动时 Follower 精确跟随",
    "所有关节运动流畅无卡顿",
    "夹爪开合功能正常",
    "摄像头画面清晰稳定",
    "带摄像头时遥操作依然流畅",
    "无异常错误或警告信息",
]

print("=" * 60)
print("完整系统测试检查清单")
print("=" * 60)
print()
for i, item in enumerate(checklist, 1):
    print(f"[ ] {i}. {item}")
print()
print("请逐项测试并确认所有功能正常！")
print("=" * 60)

### 性能基准

正常工作的系统应该达到：

| 指标 | 期望值 |
|------|--------|
| 遥操作延迟 | < 100ms |
| 位置跟随精度 | ±2° |
| 摄像头帧率 | ≥ 25 fps |
| CPU 使用率 | < 50% |
| 舵机温度 | < 60°C |

如果性能明显低于这些指标，请检查：
- USB 连接方式（避免使用 Hub）
- 系统资源占用
- 电源供电质量
- 机械摩擦阻力

---
## 🔧 常见问题

### Q1: 校准后机械臂位置不对？

**A:** 校准保存的是相对偏移，不是绝对位置。只要遥操作时跟随准确就是正常的。

### Q2: 每次重启都要重新校准吗？

**A:** 不需要。校准数据保存在 `~/.cache/calibration/` 目录下，只要：
- 使用相同的 `--robot.id` 和 `--teleop.id`
- 没有更换或重新安装舵机
- 校准文件没有被删除

就可以一直使用。

### Q3: 摄像头画面模糊或延迟？

**A:** 尝试：
1. 调整对焦（如果摄像头支持）
2. 降低分辨率（如 320x240）
3. 更换 fourcc 编码（试试 "YUYV"）
4. 使用 USB 3.0 接口

### Q4: 夹爪无法开合？

**A:** 检查：
1. 夹爪舵机 ID 是否正确 (通常是 ID 6)
2. 夹爪是否有机械卡顿
3. 重新校准夹爪

### Q5: 遥操作时出现 "Magnitude exceeds 2047"？

**A:** 这表示命令超出舵机范围：
1. 立即停止操作
2. 断电重启机械臂
3. 重新进行完整校准
4. 检查机械限位是否正确

---
## ✅ 校准和遥操作完成！

恭喜！如果所有测试都通过，说明您的 SO-ARM 系统已经正确配置。

### 下一步

完成校准后，您可以：

**02_data_collection.ipynb** - 开始采集训练数据

在数据采集中，您将学习：
- 如何录制演示动作
- 数据集格式和存储
- 数据质量检查
- 上传到 Hugging Face Hub

---

**重要提示：**

- 保存好校准文件（`~/.cache/calibration/`）
- 记住使用的 ID 名称
- 定期检查机械臂的机械状态
- 如果精度下降，重新校准

**参考资源：**
- [Seeed Studio Wiki - 校准](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/#_4)
- [Seeed Studio Wiki - 遥操作](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/#_5)